1. In this model for 2D keypoint detection, how many keypoints are being detected per image?

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, 
                               kernel_size=6, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, 
                               padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc = nn.Linear(12 * 2 * 2, 2)  
        self.tanh = nn.Tanh()

2. And this one?

In [ ]:
class SquareKPNN(nn.Module):
    def __init__(self):
        super(SquareKPNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, 
                               kernel_size=6, padding=1, device=device)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, 
                               padding=1, device=device)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc = nn.Linear(12 * 2 * 2, 8, device=device)  # Adjust the dimensions accordingly
        self.tanh = nn.Tanh()

3. For single object detection, what is this CustomLoss meassuring?

In [ ]:
class CustomLoss(nn.Module):

    def forward(self, predictions, targets):
        pred_is_object = predictions[:, 0]
        pred_box = predictions[:, 1:5]
        pred_logits = predictions[:, 5:7]

        target_is_object = targets[:, 0]
        target_box = targets[:, 1:5]
        target_class = targets[:, 5].to(torch.long)

        loss = F.mse_loss(pred_is_object, target_is_object)
        is_object = target_is_object == 1
        
        if torch.any(is_object):
            loss += F.mse_loss(pred_box[is_object], target_box[is_object])
            loss += F.cross_entropy(pred_logits[is_object], target_class[is_object])
        return loss


4. In this model, how is the image size reduced without a pooling layer?
    - What is latent_dim?

In [ ]:
encoder = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # 28x28 -> 14x14
    nn.LeakyReLU(),
    nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 14x14 -> 7x7
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(32 * 7 * 7, latent_dim),  # Latent representation
)

5. In this model, what is the Unflatten layer used for?

In [ ]:
decoder = nn.Sequential(
    nn.Linear(latent_dim, 32 * 7 * 7),
    nn.Unflatten(1, (32, 7, 7)),
    nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # 7x7 -> 14x14
    nn.LeakyReLU(),
    nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),  # 14x14 -> 28x28
    nn.Sigmoid()  # Output pixel values in range [0, 1]
)

6. What is this piece of coding doing?
    - What is the unsqueeze() method in line 6 used for?

In [ ]:
for i in range(10):
    x, _ = dataset[i]
    noise = torch.randn_like(x) * 0.1
    x = x + noise
    with torch.no_grad():
        image = model(x.unsqueeze(0).to(device))

7. In this code, what is fc_mu and fc_logvar layers used for?

In [ ]:
class VAEEncoder(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)  # 28x28 -> 14x14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)  # 14x14 -> 7x7
        self.fc_mu = nn.Linear(32 * 7 * 7, latent_dim)
        self.fc_logvar = nn.Linear(32 * 7 * 7, latent_dim)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

8. In this VAEDecoder, what is the self.fc layer used for?

In [ ]:
class VAEDecoder(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 32 * 7 * 7)
        self.conv2 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1)  # 7x7 -> 14x14
        self.conv1 = nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1)  # 14x14 -> 28x28

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 32, 7, 7)
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv1(x))
        x = torch.sigmoid(x)
        return x       



9. This code comes from a GAN network.
    - Why are there three different loss calculated?
    - Which GAN component is learning on each code fragment?
    - Why is fake.detach() in line 14?

In [ ]:
### with real data
real_images = data.to(device)
batch_size = real_images.size(0)
label = torch.full((batch_size,), REAL_LABEL, dtype=torch.float, device=device)
output = model_D(real_images).view(-1)
loss_real = loss_fn(output, label)
loss_real.backward()
conf_D_real = output.mean().item()

### with fake data
noise = torch.randn(batch_size, NOISE_SIZE, 1, 1, device=device)
fake = model_G(noise)
label.fill_(FAKE_LABEL)
output = model_D(fake.detach()).view(-1)
loss_fake = loss_fn(output, label)
loss_fake.backward()
conf_D_fake = 1 - output.mean().item()
loss_D = loss_real + loss_fake
optimizerD.step()

# Update the generator
model_G.zero_grad()
label.fill_(REAL_LABEL)  
output = model_D(fake).view(-1)
loss_G = loss_fn(output, label)
loss_G.backward()
optimizerG.step()